<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Observability for Claude Agent SDK (JS/TS) with Langfuse" sidebarTitle: "Claude Agent SDK (JS/TS)" description: "Learn how to trace Claude Agent SDK applications in JavaScript/TypeScript with Langfuse via OpenTelemetry" category: "Integrations" -->

# Integrate Langfuse with Claude Agent SDK (JS/TS)

This notebook demonstrates how to capture detailed traces from the [Claude Agent SDK](https://github.com/anthropics/claude-agent-sdk-js) in **JavaScript/TypeScript** with **[Langfuse](https://langfuse.com)** using OpenTelemetry.

> **What is Claude Agent SDK?**  
> The [Claude Agent SDK](https://platform.claude.com/docs/en/agent-sdk/overview) is Anthropic's open-source framework for building AI agents. It provides a clean API for creating tool-using agents, including native support for MCP.

> **What is Langfuse?**  
> [Langfuse](https://langfuse.com) is the open source LLM engineering platform. It provides detailed tracing, monitoring, and analytics for every prompt, model response, and tool call in your agent, making it easy to debug, evaluate, and iterate on LLM applications.

<!-- STEPS_START -->
## Step 1: Set Up Environment

Get your Langfuse API keys by signing up for [Langfuse Cloud](https://cloud.langfuse.com/) or [self-hosting Langfuse](https://langfuse.com/self-hosting). You'll also need your Anthropic API key from the [Anthropic Console](https://console.anthropic.com/).

> **Note**: This cookbook uses **Deno.js** for execution, which requires different syntax for importing packages and setting environment variables. For Node.js applications, the setup process is similar but uses standard `npm` packages and `process.env`.

In [ ]:
// Langfuse authentication keys
Deno.env.set("LANGFUSE_PUBLIC_KEY", "pk-lf-***");
Deno.env.set("LANGFUSE_SECRET_KEY", "sk-lf-***");

// Langfuse host configuration
// For US data region, set this to "https://us.cloud.langfuse.com"
Deno.env.set("LANGFUSE_BASE_URL", "https://cloud.langfuse.com");

// Set Anthropic API key
Deno.env.set("ANTHROPIC_API_KEY", "sk-ant-***");

## Step 2: Initialize OpenTelemetry with Langfuse

We use the `@langfuse/otel` package to initialize the OpenTelemetry SDK with the Langfuse span processor. This processor will automatically send all OpenTelemetry spans to Langfuse for visualization and analysis.

In [ ]:
// Import required dependencies
import 'npm:dotenv/config';
import { NodeSDK } from "npm:@opentelemetry/sdk-node";
import { LangfuseSpanProcessor } from "npm:@langfuse/otel";
 
// Export the processor to be able to flush it later
// This is important for ensuring all spans are sent to Langfuse
export const langfuseSpanProcessor = new LangfuseSpanProcessor({
    publicKey: Deno.env.get("LANGFUSE_PUBLIC_KEY")!,
    secretKey: Deno.env.get("LANGFUSE_SECRET_KEY")!,
    baseUrl: Deno.env.get("LANGFUSE_BASE_URL") ?? 'https://cloud.langfuse.com',
    environment: Deno.env.get("NODE_ENV") ?? 'development',
  });
 
// Initialize the OpenTelemetry SDK with our Langfuse processor
const sdk = new NodeSDK({
  spanProcessors: [langfuseSpanProcessor],
});
 
// Start the SDK to begin collecting telemetry
sdk.start();

console.log("OpenTelemetry SDK initialized with Langfuse");

## Step 3: Configure Claude Agent SDK Instrumentation

Use [LangSmith's Claude Agent SDK instrumentation library](https://docs.langchain.com/langsmith/trace-claude-agent-sdk) to automatically instrument the Claude Agent SDK and emit OpenTelemetry spans. This instrumentation captures:

- Model API calls with input/output content
- Tool definitions and tool call executions
- Agent turns and conversation flow
- Token usage and latency metrics

In [ ]:
import { wrapClaudeAgentSDK } from "npm:@langchain/otel-claude-agent-sdk";

// Configure the Claude Agent SDK with OpenTelemetry instrumentation
// This wraps the SDK to automatically emit spans for all agent operations
wrapClaudeAgentSDK();

console.log("Claude Agent SDK instrumentation configured");

## Step 4: Build a Hello World Agent

Let's create a simple agent with a weather tool. Every tool call and model completion will be automatically captured as an OpenTelemetry span and forwarded to Langfuse.

In [ ]:
import { Agent, tool } from "npm:@anthropic-ai/claude-agent-sdk";

// Define a simple weather tool
const getWeatherTool = tool({
  name: "get_weather",
  description: "Gets the current weather for a given city",
  parameters: {
    type: "object",
    properties: {
      city: {
        type: "string",
        description: "The city to get the weather for",
      },
    },
    required: ["city"],
  },
  execute: async ({ city }) => {
    // Simulated weather data
    const weatherData: Record<string, string> = {
      "Berlin": "Cloudy, 59°F",
      "New York": "Sunny, 75°F",
      "San Francisco": "Foggy, 62°F",
      "Tokyo": "Clear, 68°F",
    };
    
    const weather = weatherData[city] || "Weather data not available";
    return `Weather in ${city}: ${weather}`;
  },
});

// Create the agent with the weather tool
const agent = new Agent({
  model: "claude-sonnet-4-20250514",
  apiKey: Deno.env.get("ANTHROPIC_API_KEY"),
  systemPrompt: "You are a friendly travel assistant who helps with weather information. Always use the get_weather tool to check the weather for cities the user asks about.",
  tools: [getWeatherTool],
});

console.log("Agent created with weather tool");

## Step 5: Run the Agent

Now let's run the agent with a query. All interactions will be automatically traced to Langfuse.

In [ ]:
// Run the agent with a user query
const response = await agent.run("What's the weather like in Berlin and New York?");

console.log("Agent Response:");
console.log(response.content);

## Step 6: Flush Traces

To ensure all spans are sent to Langfuse before the notebook ends, we flush the span processor.

In [ ]:
// Flush the span processor to ensure all traces are sent
await langfuseSpanProcessor.forceFlush();

console.log("All traces flushed to Langfuse");

## Step 7: View the Trace in Langfuse

Head over to your **Langfuse dashboard → Traces**. You should see traces including all tool calls and model inputs/outputs.

![Claude Agent SDK example trace in Langfuse](https://langfuse.com/images/cookbook/integration_claude_agent_sdk/claude-agent-sdk-example-trace.png)

[Link to example trace in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/097f1d8982fa909b8cffb14a166ec272?timestamp=2025-12-22T10:46:15.528Z)

## Example: Adding Custom Trace Metadata

You can enrich your traces with custom metadata like user IDs, session IDs, and tags for better organization and filtering in Langfuse.

In [ ]:
import { context, trace } from "npm:@opentelemetry/api";

// Get the current tracer
const tracer = trace.getTracer("claude-agent-example");

// Create a span with custom attributes for the agent call
await tracer.startActiveSpan("weather-assistant-request", async (span) => {
  // Add Langfuse-specific attributes
  span.setAttribute("langfuse.user.id", "user-123");
  span.setAttribute("langfuse.session.id", "session-456");
  span.setAttribute("langfuse.tags", JSON.stringify(["weather", "demo"]));
  span.setAttribute("langfuse.metadata", JSON.stringify({ env: "notebook", version: "1.0" }));

  try {
    const response = await agent.run("What's the weather in Tokyo?");
    console.log("Response:", response.content);
    
    // Record successful completion
    span.setAttribute("langfuse.output", response.content);
  } catch (error) {
    span.recordException(error);
    throw error;
  } finally {
    span.end();
  }
});

// Flush to ensure the trace is sent
await langfuseSpanProcessor.forceFlush();

## Example: Streaming Responses

The Claude Agent SDK also supports streaming responses. All streaming events are automatically traced.

In [ ]:
// Run the agent with streaming
const stream = await agent.stream("Tell me about the weather in San Francisco.");

console.log("Streaming response:");
for await (const chunk of stream) {
  if (chunk.type === "text") {
    Deno.stdout.writeSync(new TextEncoder().encode(chunk.text));
  }
}
console.log("\n");

// Flush traces
await langfuseSpanProcessor.forceFlush();

<!-- STEPS_END -->

## Node.js Setup

For Node.js applications, the setup is similar. Here's how you would configure it:

```typescript
// 1. Install dependencies
// npm install @anthropic-ai/claude-agent-sdk @langfuse/otel @opentelemetry/sdk-node @langchain/otel-claude-agent-sdk

// 2. Initialize OpenTelemetry (run this before importing the Claude Agent SDK)
import { NodeSDK } from "@opentelemetry/sdk-node";
import { LangfuseSpanProcessor } from "@langfuse/otel";

const langfuseSpanProcessor = new LangfuseSpanProcessor({
  publicKey: process.env.LANGFUSE_PUBLIC_KEY!,
  secretKey: process.env.LANGFUSE_SECRET_KEY!,
  baseUrl: process.env.LANGFUSE_BASE_URL ?? "https://cloud.langfuse.com",
});

const sdk = new NodeSDK({
  spanProcessors: [langfuseSpanProcessor],
});

sdk.start();

// 3. Configure Claude Agent SDK instrumentation
import { wrapClaudeAgentSDK } from "@langchain/otel-claude-agent-sdk";
wrapClaudeAgentSDK();

// 4. Use the Claude Agent SDK as normal
import { Agent, tool } from "@anthropic-ai/claude-agent-sdk";
// ... your agent code
```

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->